In [69]:
import numpy as np
from numpy import trapz

import pandas as pd
import os
import datetime

import matplotlib

# Uses tinker as the backend for matplotlib (Not really sure what this is doing)
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from pathlib import Path

import math
from math import pi
from math import sqrt

import statistics

import scipy.integrate
from scipy.signal import find_peaks
from scipy.signal import argrelmin
from scipy.integrate import cumtrapz

In [72]:
# Local lib imports 
# from lib.cone_area_funcs import calcFFPContactArea


In [73]:
# Define global values (use very selectively!)
PI = np.pi

In [74]:

# FFP_Properties =
    
def integrateFunc(x, func, order = 1):
    # x: Dependent Variable that the derivative should be taken with respect to
    # func: function that the derivative is being taken of
    # order: the order of the integral that should be taken

    pass

def calcDerivative(x, func, order = 1, type = "forward"):
    # x: Dependent Variable that the derivative should be taken with respect to
    # func: function that the derivative is being taken of
    # order: the order of the derivative that should be taken. 1 for first derivative, 2 for second...
    # Type of derivative that should be taken (forward, backwards, central)

    pass

# def tipProperties(tipType):
#     if tipType == "cone":
#         mass = 7.71 # [kg]
#         length = 8.48 - 0.93 #[cm], originally 7.87, 7.57 fors perfect 60 deg, 8.48 measured - .93 measured 90 deg
#     if tipType == ""

    

In [75]:
# SETUP VARIABLES - USER INPUTS
BD = 3 #Bluedrop file is from 
fileNum = '01F3' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
soiltype = "c" #s = sand, c=clay, m=mixed, u=unknown
tiptype = 'c'  # c = cone, p = parabolic, b = blunt
truncate_at_cone = "n" #"y" to end analysis at a penetration depth equal to the cone length, "n" to consider full cone length
droptype = "w" #w = water, #a = air
unit_wt = "uk" #enter an effective unit weight value in kg/m^3 or "uk" if unknown 
Nkt = 10 #default value for cone factor used in su calculations
drop_finding = "m" #"m" for manual drop picking, "a" for automatic drop picking

# paste the filepath to the folder where the BD data is stored
binFilepath = Path("C:/Users/elise/Desktop/bd_manual/sample_drops")

# paste the filepath to an excel file that the analysis results will be printed in
outputPath = Path("C:/Users/elise/Desktop/bd_manual/sample_drops/Figures/Summary_Excel.xlsx")

# paste the filepath to the folder where the output figures will be saved
plotPath = Path("C:/Users/elise/Desktop/bd_manual/sample_drops/Figures")

In [105]:
from lib.BinaryFile import BinaryFile




    

In [107]:
# Get directories of all binary files in the specified directory
dir = r"C:\Geotech_Research\Field_Analysis\Bluedrop_data\Mouth 1"

import glob


column_names = ["Count", 'Unknown', "2g_accel", "18g_accel", "50g_accel", "pore_pressure", "200g_accel", 
                                                                "55g_x_tilt","55g_y_tilt", "250g_accel"]

binary_file_dirs  = get_binary_file_dirs(dir)

# Add a function to sort the binary files by date

# init list to store the data files
data_files = [None] * len(binary_file_dirs)

for i, file_dir in enumerate(binary_file_dirs):
    data_files[i] = PFFP_File(file_dir)
    data_files[i] = PFFP_File.binary_file_2_df(file_dir, column_names)    

### Things to do

* Convert the df data to SI units
* Make a way to plot the data
* Make a way to try to get the number of drops
* Convert data to bearing capacity
* Create function to assign PPFP properties
  * I don't want to do this on init because there might be surveys were you use more than one tip or bluedrop 
* Assume that each folder only contains data from one PFFP

In [117]:
import os
import glob



In [119]:
# from lib.pffpDataFolder import pffpDataFolder
calib_dir = r"C:"

folder = pffpDataFolder(dir, pffp_id=1, calibration_factor_dir=calib_dir)
folder.store_binary_files()


<built-in function dir>
[]


In [7]:
print(file)